# Libraries

In [3]:
%cd yolov5_fire/
!git clone https://github.com/Eadgnus/yolov5_fire

/content/yolov5_fire
Cloning into 'yolov5_fire'...
remote: Enumerating objects: 539, done.
remote: Counting objects: 100% (539/539), done.
remote: Compressing objects: 100% (469/469), done.
remote: Total 539 (delta 46), reused 527 (delta 40), pack-reused 0
Receiving objects: 100% (539/539), 391.35 KiB | 3.69 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [4]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00


### 캐글에서 계정 들어가서 어카운트에서 토큰발급하고 나온 kaggle.json 코랩에 넣고 실행하기

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d antrosafin/fire-dataset-in-yolo-format

fire-dataset-in-yolo-format.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip -qq '/content/fire-dataset-in-yolo-format.zip'

In [ ]:
import torch
import os
import glob
from IPython.display import Image, display

%cd yolov5_fire
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

/content/yolov5_fire
Setup complete. Using torch 2.0.1+cu118 (Tesla T4)


# Train YOLOv5 model

In [ ]:
!python train.py --img 640 --batch 16 --epochs 300 --data /content/yolov5_fire/data/fire_yaml.yaml --weights yolov5s.pt --workers 1

train: weights=yolov5s.pt, cfg=, data=/content/yolov5_fire/data/fire_yaml.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=1, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 2694 commits. Use 'git pull ultralytics master' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5 🚀 894e50ae Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cl

# 여기 이후로는 아직 안함

# Evaluate Custom YOLOv5 Detector Performance
Training losses and performance metrics are saved to Tensorboard and also to a logfile.

In [ ]:
from utils.plots import plot_results
plot_results('runs/train/exp2/results.csv')

### Prediction on Images

In [ ]:
!python detect.py --weights runs/train/exp2/weights/best.pt --img 640 --conf 0.25 --source /content/yolov5_fire/Fire_data/images/valid

detect: weights=['runs/train/exp2/weights/best.pt'], source=/content/yolov5_fire/Fire_data/images/valid, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 894e50ae Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/10 /content/yolov5_fire/Fire_data/images/valid/fire.1.png: 352x640 9 fires, 46.0ms
image 2/10 /content/yolov5_fire/Fire_data/images/valid/fire.10.png: 512x640 2 fires, 46.2ms
image 3/10 /content/yolov5_fire/Fire_data/images/valid/fire.2.png: 448x640 3 fires, 43.8ms
image 4/10 /content/yolov5_fire/Fire_data/images/valid/f

In [ ]:
#display inference on some test images
images = glob.glob('/content/yolov5_fire/runs/detect/exp2/*.png')

for imageName in images[:3]: #assuming JPG
    display(Image(filename=imageName, width=400))

Output hidden; open in https://colab.research.google.com to view.

### Prediction on Video

In [ ]:
!python detect.py --weights runs/train/exp2/weights/best.pt --img 640 --conf 0.25 --source /content/yolov5_fire/runs/train/exp2/input.mp4

detect: weights=['runs/train/exp2/weights/best.pt'], source=/content/yolov5_fire/runs/train/exp2/input.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 894e50ae Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/456) /content/yolov5_fire/runs/train/exp2/input.mp4: 384x640 2 fires, 43.6ms
video 1/1 (2/456) /content/yolov5_fire/runs/train/exp2/input.mp4: 384x640 2 fires, 7.9ms
video 1/1 (3/456) /content/yolov5_fire/runs/train/exp2/input.mp4: 384x640 2 fires, 7.8ms
video 1/1 (4/456) /content/yolov5_fire/runs/train/exp2/inpu

In [ ]:
import cv2
vidcap = cv2.VideoCapture('runs/detect/exp4/input.mp4')
success,image = vidcap.read()
images = []
while success:
    success,image = vidcap.read()
    if success:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        images.append(image)

In [ ]:
from matplotlib import animation, rc
import matplotlib.pyplot as plt

rc('animation', html='jshtml')

def create_animation(ims):
    fig = plt.figure(figsize=(9, 9))
    plt.axis('off')
    im = plt.imshow(ims[0])

    def animate_func(i):
        im.set_array(ims[i])
        return [im]

    return animation.FuncAnimation(fig, animate_func, frames = len(ims), interval = 1000//12)

create_animation(images)

# Visualize feature map

In [ ]:
image_path = "../datasets/fire/val/images/004dec94c5de631f.jpg"
display(Image(filename=image_path, width=400))

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source {image_path} --visualize

In [ ]:
display(Image(filename="runs/detect/exp3/004dec94c5de631f/stage23_C3_features.png"))